# Django ORM

   Встроенный в Django объектно-реляционный преобразователь __ORM (object-relational mapper)__ – это мощный API абстракции базы данных, который позволяет легко работать с объектами. ORM позволяет нам взаимодействовать с базой данных, используя высокоуровневые Python объекты, а не низкоуровневые SQL запросы. Это упрощает разработку и поддержку веб-приложений.

   __ORM-преобразователь__ соотносит модели с таблицами базы данных, генерирует SQL-запросы и соотносит результаты с объектами модели. ORM-преобразователь совместим с реляционными системами управления базами данных __MySQL, PostgreSQL, SQLite, Oracle и MariaDB__.

Базу данных проекта можно определять в настроечном параметре __DATABASES__ файла __settings.py__ проекта. Django может работать с несколькими базами данных одновременно, при этом можно программировать маршрутизаторы баз данных, чтобы создавать конкретно-прикладные схемы маршрутизации данных.

Встроенный в Django ORM-преобразователь основан на итерируемых наборах запросов QuerySet. __Итерируемый набор запросов QuerySet__ – это коллекция запросов к базе данных, предназначенных для извлечения объектов из базы данных. К наборам запросов можно применять фильтры, чтобы сужать результаты запросов на основе заданных параметров.

## Создание объектов

Откройте терминал в корневой директории вашего проекта Django (на уровне с settings.py) и введите команду:

In [ ]:
python manage.py shell

Это откроет интерактивную оболочку Python с загруженным Django окружением.

Наберите следующие ниже строки:

In [ ]:
from django.contrib.auth.models import User

new_user = User.objects.create_user('username', 'email@example.com', 'password')
new_user.first_name = 'Ivan'
new_user.last_name = 'Ivanov'
new_user.save()

Проанализируем работу приведенного выше кода:
   * Импортируется модель __User__ из __django.contrib.auth.models__ для работы с ней.
   * Создается новый пользователя с указанными атрибутами. В Django для создания пользователя лучше использовать метод __create_user__, который обрабатывает некоторые важные аспекты, такие как хэширование пароля.
   * __new_user.save()__ сохраняет объект __User__ в базе данных, используя метод __save()__.

Пользователя можно получить из базы данных и вывести информацию:

In [ ]:
user = User.objects.get(username='username')
print(user.first_name, user.last_name, user.email)

Этот код нам выведет внутри командной оболочки следующее:

__Ivan Ivanov email@example.com__

Метод __get()__ позволяет извлекать из базы данных только один объект.Этот метод ожидает результат, совпадающий с запросом. <br></br>Если база данных не возвращает результатов, то указанный метод вызовет исключение __DoesNotExist__, а если база данных возвращает более одного результата, то он вызовет исключение __MultipleObjectsReturned__. Оба исключения являются атрибутами модельного класса, на котором выполняется запрос.

Импортируем модели Project и Task.:

In [ ]:
from tasks.models import Project, Task

Наберите следующие ниже строки:

In [ ]:
project = Project(name="Web Development", description="Web Development Project")

Мы создаем экземпляр класса Project с конкретным названием и описанием. Этот объект находится в памяти и не сохраняется в базе данных; мы создали объект Python, который можно использовать на стадии работы программы, но который не сохраняется в базе данных. Наконец, мы сохраняем объект Project в базе данных, используя метод __save()__:

In [ ]:
project.save()

Приведенное выше действие за кулисами выполняет инструкцию SQL __INSERT__.

Аналогично создадим объект Task:

In [ ]:
task = Task(
    project=project,
    name="Design Landing Page",
    description="Create a responsive landing page",
    assignee=user,
    status='New'
)
task.save()

## Обновление объектов

Измените название задачи на что-то другое и снова сохраните объект:

In [ ]:
task.name="Redesign Page"
task.save()

На этот раз метод save() исполняет инструкцию SQL __UPDATE__.

## Извлечение объектов

   Одиночный объект извлекается из базы данных методом __get()__. Мы применили этот метод посредством метода __User.objects.get()__. Каждая модель Django имеет по меньшей мере один модельный менеджер, а менеджер, который применяется по умолчанию, называется __objects__. Набор запросов __QuerySet__ можно получать с помощью модельного менеджера.
Для того чтобы извлечь все объекты из таблицы, используется метод __all()__ применяемого по умолчанию менеджера __objects__.

In [ ]:
all_projects = Project.objects.all()

   Обратите внимание, что этот __QuerySet__ еще не исполнен.<br></br>Наборы запросов QuerySet в Django являются ленивыми, то есть они вычисляются только тогда, когда это приходится делать. Подобное поведение придает наборам запросов QuerySet большую эффективность. Если не назначать набор запросов QuerySet переменной, а вместо этого писать его непосредственно в оболочке Python, то инструкция SQL набора запросов будет исполняться, потому что вы побуждаете ее генерировать результат:

In [ ]:
Project.objects.all()

__<QuerySet [<Project: Web Development>]>__

Для просмотра всех проектов и задач используем следующий код:

In [ ]:
all_projects = Project.objects.all()
all_tasks = Task.objects.all()
for project in all_projects:
    print(project)
for task in all_tasks:
    print(task)

Для удаления объектов используется метод __delete()__:

In [ ]:
task.delete()

### Другие методы запросов в Django ORM

1. Метод __filter()__:<br></br>
Метод __filter()__ применяется для получения набора записей из базы данных, соответствующих определённым критериям. Этот метод возвращает QuerySet, который можно далее модифицировать или оценивать.<br></br>
   * Фильтрация задач по статусу:<br></br>
   Здесь мы получаем все задачи со статусом 'В работе'.

In [ ]:
in_progress_tasks = Task.objects.filter(status='In_progress')
for task in in_progress_tasks:
    print(task.name, task.project.name)


   * Фильтрация проектов по дате создания:<br></br>
   Если мы хотим найти проекты, созданные после определенной даты:

In [ ]:
from datetime import datetime

date_threshold = datetime(2022, 1, 1)
recent_projects = Project.objects.filter(created_at__gt=date_threshold)
for project in recent_projects:
    print(project.name, project.created_at)

Здесь __created_at\_\_gt__ означает "больше, чем (gt - greater than) указанная дата".

2. Метод __exclude()__:<br></br>
Исключает записи, соответствующие определённым критериям.

In [ ]:
non_completed_tasks = Task.objects.exclude(status='Completed')

3. Метод __order_by()__:<br></br>
Упорядочивает QuerySet по определённому полю.

In [ ]:
tasks_ordered_by_creation = Task.objects.order_by('created_at')

4. Метод __count()__:<br></br>
Возвращает количество объектов, соответствующих запросу.

In [ ]:
count_completed_tasks = Task.objects.filter(status='Completed').count()

5. Метод __exists()__:<br></br>
Возвращает __True__ или __False__ в зависимости от существуют ли в базе данных записи, соответствующие заданным критериям. 

In [ ]:
from tasks.models import Task

if Task.objects.filter(status='New').exists():
    print("Есть новые задачи")
else:
    print("Новых задач нет")

Если вам нужно работать с объектами, которые вы ищете (например, вывести их на экран или обновить), использование __exists()__ может быть неэффективным, так как вам всё равно придется выполнить дополнительный запрос для получения самих объектов. В таких случаях лучше сразу использовать __filter(), get()__ или другие методы QuerySet'ов, которые возвращают сами объекты.

6. Методы __aggregate()__ и __annotate()__:<br></br>
Используются для выполнения сложных агрегирующих запросов, таких как подсчёт, суммирование, средние значения.

Метод __aggregate()__ применяется к QuerySet и используется для выполнения агрегатных функций на всем наборе данных, возвращая словарь с результатами вычислений.<br></br>
Представим, что у вас есть модель __Sale__ с полями __amount__ и __date__. Вы хотите вычислить общую сумму продаж.

In [ ]:
from django.db.models import Sum
from .models import Sale

total_sales = Sale.objects.aggregate(total=Sum('amount'))
print(total_sales['total'])

Здесь __aggregate(total=Sum('amount')__ вычисляет сумму всех значений поля amount во всех записях Sale.

Метод __annotate()__ похож на __aggregate()__, но вместо вычисления агрегатных значений для всего QuerySet, он добавляет агрегатные значения к каждому объекту в QuerySet.<br></br>
Предположим, у вас есть модели __Author__ и __Book__, где каждая книга связана с автором. Вы хотите получить список авторов с указанием общего числа их книг.

In [ ]:
from django.db.models import Count
from .models import Author

authors = Author.objects.annotate(num_books=Count('book'))
for author in authors:
    print(f"{author.name} написал {author.num_books} книг(и)")

Здесь __annotate(num_books=Count('book'))__ добавляет к каждому объекту __Author__ дополнительное поле __num_books__, которое содержит количество связанных с автором книг.

## Класс Q в Django

Класс __Q__ в __Django ORM__ используется для выполнения сложных запросов с использованием логических операторов (__AND, OR, NOT__). Класс __Q__ полезен, когда необходимо выполнить запросы с условиями, которые сложно или невозможно выразить с использованием стандартных методов QuerySet, таких как __filter()__ и __exclude()__.

Примеры запросов, использующих | (OR) и & (AND):

In [ ]:
from django.db.models import Q
from .models import MyModel

# Пример OR
queryset = MyModel.objects.filter(Q(field1='value1') | Q(field2='value2'))

# Пример AND
queryset = MyModel.objects.filter(Q(field1='value1') & Q(field3='value3'))

Сложные запросы с использованием вложенных выражений Q:

In [ ]:
queryset = MyModel.objects.filter(
    Q(field1='value1') | (Q(field2='value2') & ~Q(field4='value4'))
)

__~Q__ "инвертирует" логику запроса. Это аналогично использованию __NOT__ в SQL.

In [ ]:
from django.db.models import Q
from .models import Task

# Получить все задачи, которые не завершены
tasks = Task.objects.filter(~Q(status='Completed'))

## Класс F и Value в Django

Класс __F__ представляет значение поля модели внутри запроса, он позволяет выполнить операции с полями модели без явного извлечения их из базы данных.

   * Обновление поля:<br></br>
   Инкрементирование значения поля:

In [ ]:
from django.db.models import F 
from tasks.models import Task

Task.objects.filter(status='In_progress').update(priority=F('priority') + 1)

   * Сравнение полей одного объекта:<br></br>
   Выбор задач, где дата обновления позже даты создания:

In [ ]:
recent_tasks = Task.objects.filter(updated_at__gt=F('created_at'))

Выбор задач, где название проекта совпадает с именем исполнителя:

In [ ]:
Task.objects.filter(project__name=F('assignee__username'))

Класс __Value__ используется для включения литеральных значений или выражений в запросы Django ORM.

   * Добавление статического значения в запрос:

In [ ]:
from django.db.models import Value
from django.db.models.functions import Concat
from tasks.models import Task

Task.objects.annotate(full_description=Concat(Value('Task: '), 'description'))

   * Обновление статуса задач на основе динамического условия:

In [ ]:
Task.objects.filter(updated_at__gt=F('created_at')).update(status=Value('Completed'))